In [8]:
%matplotlib inline
import numpy as np
import pandas as pd

In [14]:
SAMPLE_SIZE = 1000

A = np.random.normal(0, 0.8, SAMPLE_SIZE)
B = np.random.normal(1, 1, SAMPLE_SIZE)
C = A**3 + B**2 + np.random.normal(0,1, SAMPLE_SIZE)
D = C**3 + np.random.normal(0,1, SAMPLE_SIZE)
E = np.tan(C) + np.random.normal(0,1, SAMPLE_SIZE)
F = D**0.5 + np.sin(E) + np.random.normal(0,1, SAMPLE_SIZE)
G = F**0.5 + np.random.normal(0,1, SAMPLE_SIZE)

/Users/tanmayee/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in sqrt
  
/Users/tanmayee/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in sqrt
  if __name__ == '__main__':


In [11]:
from pygam import LinearGAM, s, f

In [34]:
gam = LinearGAM(s(0) + s(1)).gridsearch(np.hstack((A.reshape(1000, 1), B.reshape(1000, 1))), C)

100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [35]:
gam.summary()

LinearGAM                                                                                                 
=============================================== ==========================================================
Distribution:                        NormalDist Effective DoF:                                     13.8186
Link Function:                     IdentityLink Log Likelihood:                                 -1430.4482
Number of Samples:                         1000 AIC:                                             2890.5337
                                                AICc:                                              2891.01
                                                GCV:                                                1.0628
                                                Scale:                                              1.0364
                                                Pseudo R-Squared:                                   0.8978
Feature Function                  Lam

/Users/tanmayee/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: KNOWN BUG: p-values computed in this summary are likely much smaller than they should be. 
 
Please do not make inferences based on these values! 

Collaborate on a solution, and stay up to date at: 
github.com/dswah/pyGAM/issues/163 

  """Entry point for launching an IPython kernel.
